In [10]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint

In [11]:
#test = pd.read_csv('../../input/test.csv')

In [12]:
#test.describe()

In [13]:
#train = pd.read_csv('../../input/train.csv')

In [14]:
#train.describe()

In [15]:
FE.ReadHdf('both')

NameError: name 'FE' is not defined

In [43]:
FE.train.apply(lambda x: x.distinct())

AttributeError: ("'Series' object has no attribute 'distinct'", u'occurred at index Semana')

In [44]:
FE.train.apply(lambda x:x.nunique())

Semana                        7
Agencia_ID                  552
Canal_ID                      9
Ruta_SAK                   3603
Cliente_ID               880604
Producto_ID                1799
Venta_uni_hoy              2116
Venta_hoy                 73515
Dev_uni_proxima             558
Dev_proxima               14240
Demanda_uni_equil          2091
log_Demanda_uni_equil      2091
dtype: int64

In [6]:
FE.train.loc[:,'log_Demanda_uni_equil'] =  FE.train.loc[:,'Demanda_uni_equil'].apply(lambda x:round(np.expm1(x))).astype('uint32')

In [10]:
train = JoinWithAggregate(FE.train,['Semana','Producto_ID','Cliente_ID'])

Index([u'count'], dtype='object')


In [18]:
train.head(5)

,Semana,Producto_ID,Cliente_ID,log_Demanda_uni_equil
10006212,3,53,1216931,116
10006225,3,53,1216931,540
10006238,3,53,1216931,309
12553334,4,53,1216931,737
12553344,4,53,1216931,184


In [13]:
train = train[['Semana','Producto_ID','Cliente_ID','log_Demanda_uni_equil']]

In [19]:
groupby1st = train.groupby(["Semana","Producto_ID","Cliente_ID"]).agg(["count","mean","sum"])
groupby1st.columns =  groupby1st.columns.droplevel(0)
groupby1st = groupby1st.reset_index()
groupby1st.drop("Semana",axis=1, inplace=True)
groupby2nd = groupby1st.groupby(["Producto_ID","Cliente_ID"]).agg(["count","mean","sum"])

In [21]:
groupby2nd.columns = groupby2nd.columns.droplevel(0)
groupby2nd = groupby2nd.iloc[:,[4,7]]

In [22]:
groupby2nd[:10]

mean         mean
Producto_ID Cliente_ID                         
53          1216931     422.247619  1406.285714
            2502084     397.027778  1572.333333
72          39220         5.833333    11.666667
            89365         6.500000    13.000000
            294060        4.000000     8.000000
            330744        3.500000     7.000000
            516827        1.500000     3.000000
            613549        5.000000    10.000000
            1229789       4.500000     9.000000
            2220391       3.000000     6.000000

In [24]:
merged = FE.test.merge(groupby2nd, right_index = True, left_on= ["Producto_ID","Cliente_ID"], how ="left")

In [36]:
names = list( merged.columns)
names[8] = "meanOfSum"
names
merged.columns = names

In [37]:
merged[:5]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,mean,meanOfSum
0,0,11,4037,1,2209,4639078,35305,NaN,NaN
1,1,11,2237,1,1226,4705135,1238,NaN,NaN
2,2,10,2045,1,2831,4549769,32940,NaN,NaN
3,3,11,1227,1,4448,4717855,43066,NaN,NaN
4,4,11,1219,1,1130,966351,1277,NaN,NaN


In [46]:
tempMerge = merged

In [ ]:
merged.fillna(0,inplace=True)

In [54]:
merged[  merged['mean']!=0 ][:3]

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,mean,meanOfSum
1799,1799,11,1910,11,3960,4434754,32962,37.000000,74.000000
2664,2664,10,1945,1,8017,4284011,43069,4.500000,9.000000
2730,2730,10,1571,5,3009,653378,43070,198.484869,37351.714286


In [56]:
SubmissionMean = merged[["id","mean"]]
SubmissionMean[:3]

,id,mean
0,0,0
1,1,0
2,2,0


In [57]:
SubmissionSum = merged[["id","meanOfSum"]]
SubmissionSum[:3]

,id,meanOfSum
0,0,0
1,1,0
2,2,0


In [63]:
SubmissionMean[  SubmissionMean['Demanda_uni_equil']!=0 ][:3]

,id,Demanda_uni_equil
1799,1799,37.000000
2664,2664,4.500000
2730,2730,198.484869


In [59]:
SubmissionSum[  SubmissionSum['meanOfSum']!=0 ][:3]

,id,meanOfSum
1799,1799,74.000000
2664,2664,9.000000
2730,2730,37351.714286


In [61]:
SubmissionSum.columns = ["id","Demanda_uni_equil"]
SubmissionMean.columns = ["id","Demanda_uni_equil"]

In [65]:
SubmissionMean[  SubmissionMean['Demanda_uni_equil']!=0 ][:3]

,id,Demanda_uni_equil
1799,1799,37.000000
2664,2664,4.500000
2730,2730,198.484869


In [66]:
SubmissionSum[  SubmissionSum['Demanda_uni_equil']!=0 ][:3]

,id,Demanda_uni_equil
1799,1799,74.000000
2664,2664,9.000000
2730,2730,37351.714286


In [67]:
SubmissionSum.to_csv('../../submissions/submissionSum.csv', index= False)

In [68]:
SubmissionMean.to_csv('../../submissions/submissionMean.csv', index= False)

In [55]:
merged[['id','mean']].to_csv(index= False)

,id,mean
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [ ]:
submissionMean = merged[['id','meanOfSum']]

In [ ]:
groupby1st = testDF.groupby(["Semana","Producto_ID","Cliente_ID"]).agg(["count","mean","sum"])
groupby1st.columns =  groupby1st.columns.droplevel(0)
groupby1st = groupby1st.reset_index()
groupby1st.drop("Semana",axis=1, inplace=True)
groupby2nd = groupby1st.groupby(["Producto_ID","Cliente_ID"]).agg(["count","mean","sum"])

In [14]:
train = JoinWithAggregate(FE.train,['Semana','Producto_ID','Cliente_ID'])

Index([u'count'], dtype='object')


In [ ]:
investigate.loc[:,'unitSalesPrice'] =  investigate['Venta_hoy']/investigate['Venta_uni_hoy']
investigate.head(2)

In [ ]:
investigate.drop('count',axis=1,inplace=True)

In [ ]:
PC = investigate
gc.collect

In [ ]:
PC.count()

In [ ]:
PCA = JoinWithAggregate(FE.train,['Semana','Producto_ID','Cliente_ID','Agencia_ID'])

In [ ]:
PCA.count()

In [ ]:
PCR = JoinWithAggregate(FE.train,['Semana','Producto_ID','Cliente_ID','Ruta_SAK'])

In [ ]:
PCR.count()

In [59]:
groupCount = FE.train.loc[:,['Semana','Producto_ID','Cliente_ID','Agencia_ID','Canal_ID']+
              ['Demanda_uni_equil']].groupby(['Semana','Producto_ID','Cliente_ID','Agencia_ID','Canal_ID']).count()
groupCount[groupCount['Demanda_uni_equil']>1].count()

Demanda_uni_equil    32599
dtype: int64

In [60]:
groupCount = FE.train.loc[:,['Semana','Producto_ID','Cliente_ID','Ruta_SAK','Canal_ID']+
              ['Demanda_uni_equil']].groupby(['Semana','Producto_ID','Cliente_ID','Ruta_SAK','Canal_ID']).count()
groupCount[groupCount['Demanda_uni_equil']>1].count()

Demanda_uni_equil    15584
dtype: int64

In [6]:
FE.train.head(5)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0,1.386294
1,3,1110,7,3301,15766,1216,4,33.520000,0,0,1.609438
2,3,1110,7,3301,15766,1238,4,39.320000,0,0,1.609438
3,3,1110,7,3301,15766,1240,4,33.520000,0,0,1.609438
4,3,1110,7,3301,15766,1242,3,22.920000,0,0,1.386294


In [61]:
train = FE.train

In [63]:
train.loc[:,'Log_Demanda_uni_equil'] =  train.loc[:,'Demanda_uni_equil'].apply(np.expm1).astype('uint16')
train.head(2)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Log_Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0,1.386294,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0,1.609438,4


In [80]:
#filtered = train.loc[:,['Semana','Producto_ID','Cliente_ID','Demanda_uni_equil']].groupby(
filtered = train.groupby(
    ['Semana','Producto_ID','Cliente_ID']).count()
filtered
#filtered = filtered[filtered.loc[:,'Demanda_uni_equil']>1]
#filtered.columns = ['count']

Agencia_ID  Canal_ID  Ruta_SAK  Venta_uni_hoy  \
Semana Producto_ID Cliente_ID                                                  
3      41          681747               1         1         1              1   
                   684023               1         1         1              1   
                   685079               1         1         1              1   
                   1035265              1         1         1              1   
                   1546790              1         1         1              1   
                   1623763              1         1         1              1   
                   2482890              1         1         1              1   
                   4586525              1         1         1              1   
       53          26372                1         1         1              1   
                   36161                1         1         1              1   
                   1216931              3         3         3              3   
                   3476372              1         1         1              1   
       72          2025                 1         1         1              1   
                   2029                 1         1         1              1   
                   2100                 1         1         1              1   
                   2154                 1         1         1              1   
                   2187                 1         1         1              1   
                   2408                 1         1         1              1   
                   2527                 1         1         1              1   
                   2541                 1         1         1              1   
                   2610                 1         1         1              1   
                   3212                 1         1         1              1   
                   3314                 1         1         1              1   
                   3334                 1         1         1              1   
                   3373                 1         1         1              1   
                   3383                 1         1         1              1   
                   3394                 1         1         1              1   
                   3409                 1         1         1              1   
                   3431                 1         1         1              1   
                   3473                 1         1         1              1   
...                                   ...       ...       ...            ...   
9      49994       2405293              1         1         1              1   
                   2412968              1         1         1              1   
                   2444219              1         1         1              1   
                   2485404              1         1         1              1   
                   2498105              1         1         1              1   
                   2501972              1         1         1              1   
                   2501973              1         1         1              1   
                   4231009              1         1         1              1   
                   4238474              1         1         1              1   
                   4284395              1         1         1              1   
                   4289316              1         1         1              1   
                   4305886              1         1         1              1   
                   4316312              1         1         1              1   
                   4335667              1         1         1              1   
                   4387348              1         1         1              1   
                   4465453              1         1         1              1   
                   4514418              1         1         1              1   
                   4518639     

In [82]:
filtered = filtered[filtered.loc[:,'Demanda_uni_equil']>1]

In [83]:
filtered.head(10)

Agencia_ID  Canal_ID  Ruta_SAK  Venta_uni_hoy  \
Semana Producto_ID Cliente_ID                                                  
3      53          1216931              3         3         3              3   
       72          1229789              2         2         2              2   
                   4331893              2         2         2              2   
       73          58881                2         2         2              2   
                   77832                2         2         2              2   
                   148432               2         2         2              2   
                   880294               2         2         2              2   
                   2501968              2         2         2              2   
                   3213810              2         2         2              2   
                   3553927              2         2         2              2   

                               Venta_hoy  Dev_uni_proxima  Dev_proxima  \
Semana Producto_ID Cliente_ID                                            
3      53          1216931             3                3            3   
       72          1229789             2                2            2   
                   4331893             2                2            2   
       73          58881               2                2            2   
                   77832               2                2            2   
                   148432              2                2            2   
                   880294              2                2            2   
                   2501968             2                2            2   
                   3213810             2                2            2   
                   3553927             2                2            2   

                               Demanda_uni_equil  Log_Demanda_uni_equil  
Semana Producto_ID Cliente_ID                                            
3      53          1216931                     3                      3  
       72          1229789                     2                      2  
                   4331893                     2                      2  
       73          58881                       2                      2  
                   77832                       2                      2  
                   148432                      2                      2  
                   880294                      2                      2  
                   2501968                     2                      2  
                   3213810                     2                      2  
                   3553927                     2                      2

In [9]:
FE.ReadHdf('test')

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


id             uint32
Semana          uint8
Agencia_ID     uint16
Canal_ID        uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999251 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 133.5 MB
None


In [13]:
investigate.loc[:,'Log_Demanda_uni_equil'] =  investigate.loc[:,'Demanda_uni_equil'].apply(np.expm1).astype('uint16')

In [14]:
investigate.head(5)

,count,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Log_Demanda_uni_equil
10006212,3,3,2655,4,6654,1216931,53,116,1658.800049,0,0,4.762174,116
10006225,3,3,2655,4,6655,1216931,53,540,7722.000000,0,0,6.293419,540
10006238,3,3,2655,4,6656,1216931,53,309,4418.700195,0,0,5.736572,309
210823,2,3,1116,1,1609,1229789,72,4,14.800000,0,0,1.609438,4
566442,2,3,1121,1,1619,1229789,72,5,18.500000,0,0,1.791759,5


In [86]:
filtered.reset_index().head(5)

,Semana,Producto_ID,Cliente_ID,Agencia_ID,Canal_ID,Ruta_SAK,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Log_Demanda_uni_equil
0,3,53,1216931,3,3,3,3,3,3,3,3,3
1,3,72,1229789,2,2,2,2,2,2,2,2,2
2,3,72,4331893,2,2,2,2,2,2,2,2,2
3,3,73,58881,2,2,2,2,2,2,2,2,2
4,3,73,77832,2,2,2,2,2,2,2,2,2


In [19]:
merged = investigate.reset_index().merge(FE.test, on= ['Producto_ID','Cliente_ID'], how ="inner")

In [28]:
merged = merged.set_index(['Semana_y','Cliente_ID','Producto_ID'])

KeyError: 'Semana_y'

In [29]:
merged = merged.sort_index()

In [30]:
merged[0:20]

index  count  Semana_x  Agencia_ID_x  \
Semana_y Cliente_ID Producto_ID                                            
10       60         34348        31625799      4         5          2655   
                    34348        31625799      4         5          2655   
                    34348        31625801      4         5          2655   
                    34348        31625801      4         5          2655   
                    34348        31625804      4         5          2655   
                    34348        31625804      4         5          2655   
                    34348        31625806      4         5          2655   
                    34348        31625806      4         5          2655   
                    34348        41839017      5         6          2655   
                    34348        41839017      5         6          2655   
                    34348        41839020      5         6          2655   
                    34348        41839020      5         6          2655   
                    34348        41839023      5         6          2655   
                    34348        41839023      5         6          2655   
                    34348        41839027      5         6          2655   
                    34348        41839027      5         6          2655   
                    34348        41839030      5         6          2655   
                    34348        41839030      5         6          2655   
                    34348        52243815      3         7          2655   
                    34348        52243815      3         7          2655   

                                 Canal_ID_x  Ruta_SAK_x  Venta_uni_hoy  \
Semana_y Cliente_ID Producto_ID                                          
10       60         34348                 2        4120            192   
                    34348                 2        4120            192   
                    34348                 2        4122            192   
                    34348                 2        4122            192   
                    34348                 2        4126            384   
                    34348                 2        4126            384   
                    34348                 2        4130            192   
                    34348                 2        4130            192   
                    34348                 2        4120            192   
                    34348                 2        4120            192   
                    34348                 2        4121            192   
                    34348                 2        4121            192   
                    34348                 2        4126            192   
                    34348                 2        4126            192   
                    34348                 2        4128            192   
                    34348                 2        4128            192   
                    34348                 2        4130            192   
                    34348                 2        4130            192   
                    34348                 2        4121            192   
                    34348                 2        4121            192   

                                    Venta_hoy  Dev_uni_proxima  Dev_proxima  \
Semana_y Cliente_ID Producto_ID                                               
10       60         34348        14962.559570                0            0   
                    34348        14962.559570                0            0   
                    34348        14962.559570                0            0   
                    34348        14962.559570                0            0   
                    34348        29925.119141                0            0   
                    34348        29925.119141                0            0   
                    34348        14962.559570                0            0   
                    343

In [6]:
def JoinWithAggregate(df, columnList):
    filtered = df.loc[:,columnList+['Demanda_uni_equil']].groupby(columnList).count()
    filtered = filtered[filtered.loc[:,'Demanda_uni_equil']>1]
    filtered.columns = ['count']
    print filtered.columns
    gc.collect()
    return filtered.merge(FE.train,left_index = True, right_on= columnList, how ="left")

In [16]:
class FeatureEngineering:
    #ValidationStart = 0
    #lag = 0
    #trainHdfPath = ""
    #trainHdfFile = ""
    #testHdfPath = ""
    #testHdfFile = ""
    #train = pd.DataFrame()
    #test = pd.DataFrame()
    
    def __init__(self, ValidationStart, ValidationSmallerThan, trainHdfPath, trainHdfFile, testHdfPath, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationSmallerThan = ValidationSmallerThan
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath = testHdfPath
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def __changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        data.index = data.index.astype('uint32')
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"
            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_hdf(self.testHdfPath,self.testHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"  
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            #__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            #__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)

    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
                self.train.loc[:,column] =  self.train[column].apply(func).astype(localType)
            elif trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
                self.test.loc[:,column] =  self.test[column].apply(func).astype(localType)
            
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t')
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test.to_hdf(self.testHdfPath, self.testHdfFile, format='t')
        
    def AddDemandaGeneralMean(self):
        DemandaMeanWithoutLag = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart]
        self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        display(self.train)
        
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        lag = 0 if config.lag == 0 else config.lag-1
        tempData = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart - lag]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData.loc[:,'Semana'].apply(lambda x:x + config.lag)
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
        
        for name,groups in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)
                
                ##First, select columns not to fill memory..then groupby..
                
                groupedDataframe =  pd.DataFrame({name:tempData[groups+['Demanda_uni_equil']].groupby(groups)
                    ['Demanda_uni_equil'].mean().astype('float32')})
                #join..
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                                  right_index=True, how='left', sort=False,copy=False)
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable]), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]
                count = self.train.loc[self.train['Semana'] >= self.ValidationStart,config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                gc.collect()
                if count == 0:
                    break
        display(self.train)        
        gc.collect()
        return 
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train.loc[self.train.loc[:,'Semana']>= 3 + self.maxLag]
        display(self.train)

In [17]:
parameterDict =       {"ValidationStart":6, 
 "ValidationSmallerThan":7,
   "maxLag":2,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'trainCsvPath': '../../input/train.csv', 'testHdfPath': '../../input/test.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 6, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5', 'ValidationSmallerThan': 7}


In [18]:
FE.ReadHdf('both')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0,1.386294
1,3,1110,7,3301,15766,1216,4,33.520000,0,0,1.609438


Semana                 uint8
Agencia_ID            uint16
Canal_ID               uint8
Ruta_SAK              uint16
Cliente_ID            uint32
Producto_ID           uint16
Venta_uni_hoy         uint16
Venta_hoy            float32
Dev_uni_proxima       uint32
Dev_proxima          float32
Demanda_uni_equil    float32
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    float32
dtypes: float32(3), uint16(4), uint32(2), uint8(2)
memory usage: 2.3 GB
None


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238


id             uint32
Semana          uint8
Agencia_ID     uint16
Canal_ID        uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6999251 entries, 0 to 6999250
Data columns (total 7 columns):
id             uint32
Semana         uint8
Agencia_ID     uint16
Canal_ID       uint8
Ruta_SAK       uint16
Cliente_ID     uint32
Producto_ID    uint16
dtypes: uint16(3), uint32(2), uint8(2)
memory usage: 133.5 MB
None


In [19]:
FE.train.loc[:,'log_Demanda_uni_equil'] =  FE.train.loc[:,'Demanda_uni_equil'].apply(lambda x:round(np.expm1(x))).astype('uint32')

In [20]:
FE.train.drop("Demanda_uni_equil",axis=1,inplace=True)

In [21]:
FE.train[:2]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,log_Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0,4


## demanda0

In [8]:
demanda0 = FE.train[FE.train.log_Demanda_uni_equil==0]

In [10]:
demanda0 = demanda0.sort(columns=["Producto_ID","Cliente_ID"])

In [50]:
demanda0.drop("Demanda_uni_equil",axis=1,inplace=True)

In [51]:
demanda0[:3]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,log_Demanda_uni_equil
41695437,6,2281,7,3306,465617,41,0,0.000000,30,543.900024,0
62493519,8,2281,7,3306,465617,41,30,543.900024,70,1269.099976,0
9844037,3,2281,7,3306,685079,41,0,0.000000,9,163.169998,0


In [ ]:
3306	465617

In [65]:
P41 =  FE.train.loc[ (FE.train.Producto_ID == 73)  ].sort(["Producto_ID","Cliente_ID","Semana"])

In [66]:
P41[:50]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,log_Demanda_uni_equil
70665470,9,2012,1,1059,2025,73,1,21.320000,0,0.000000,1
7500469,3,2012,1,1059,2061,73,5,106.599998,0,0.000000,5
18459877,4,2012,1,1059,2061,73,1,21.320000,0,0.000000,1
49875036,7,2012,1,1059,2061,73,7,149.240005,0,0.000000,7
60224470,8,2012,1,1059,2061,73,4,85.279999,0,0.000000,4
70665485,9,2012,1,1059,2061,73,3,63.959999,0,0.000000,3
60224486,8,2012,1,1059,2073,73,1,21.320000,0,0.000000,1
29123230,5,2012,1,1055,2093,73,1,21.320000,0,0.000000,1
39479868,6,2012,1,1055,2093,73,3,63.959999,0,0.000000,3
49872867,7,2012,1,1055,2093,73,1,21.320000,0,0.000000,1


## demanda0VentaUniHoy1

In [ ]:
demanda0VentaUniHoy1 = FE.train[(FE.train.log_Demanda_uni_equil==0)&(FE.train.Venta_uni_hoy>0)]

In [24]:
demanda0VentaUniHoy1 = demanda0VentaUniHoy1.sort(columns=["Producto_ID","Cliente_ID"])

In [33]:
print demanda0VentaUniHoy1["Semana"].count()
demanda0VentaUniHoy1[:5]

1034262


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,log_Demanda_uni_equil
62493519,8,2281,7,3306,465617,41,30,543.900024,70,1269.099976,0,0
55398938,8,1227,1,1614,8047,72,1,3.700000,3,11.100000,0,0
65812335,9,1227,1,1617,12277,72,2,7.400000,3,11.100000,0,0
7567893,3,2012,1,5010,13206,72,5,18.500000,12,44.400002,0,0
7663475,3,2013,1,5003,14910,72,4,14.800000,8,29.600000,0,0


In [1]:
demanda0VentaUniHoy1.groupby(["Producto_ID"]).count()

NameError: name 'demanda0VentaUniHoy1' is not defined

## demanda1Venta1Dev1

In [26]:
demanda1Venta1Dev1 = FE.train[(FE.train.log_Demanda_uni_equil>1)&(FE.train.Venta_uni_hoy>1)&(FE.train.Dev_uni_proxima>1)]

In [27]:
demanda1Venta1Dev1 = demanda1Venta1Dev1.sort(columns=["Producto_ID","Cliente_ID"])

In [ ]:
demanda1Venta1Dev1.drop("Demanda_uni_equil",axis=1,inplace=True)

In [36]:
demanda1Venta1Dev1[:5]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,log_Demanda_uni_equil
11043774,3,23879,7,3201,4586525,41,20,362.600006,5,90.650002,15
22052708,4,23879,7,3201,4586525,41,7,126.910004,3,54.389999,4
39552699,6,2012,1,5012,3897,72,5,18.500000,3,11.100000,2
60388158,8,2013,1,5004,6224,72,6,22.200001,4,14.800000,2
18630137,4,2013,1,5006,8111,72,10,37.000000,8,29.600000,2


In [117]:
demanda1Venta1Dev1.groupby(["Producto_ID"]).count()[:5]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,log_Demanda_uni_equil
Producto_ID,,,,,,,,,,
41,2,2,2,2,2,2,2,2,2,2
72,658,658,658,658,658,658,658,658,658,658
73,399,399,399,399,399,399,399,399,399,399
106,20,20,20,20,20,20,20,20,20,20
123,68,68,68,68,68,68,68,68,68,68


## Check if Venta - Dev = Demanda
* There 615.000 record which is not equal..
* But there are 0 record demand is not equal to difference of them and also demanda > 0!!..

In [31]:
#DemandaNotEqualVentaSubstractDev = FE.train[(FE.train.log_Demanda_uni_equil != 
 #   FE.train.Venta_uni_hoy-FE.train.Dev_uni_proxima) & (FE.train.log_Demanda_uni_equil > 0)]
DemandaNotEqualVentaSubstractDev = FE.train[(FE.train.log_Demanda_uni_equil != 
    FE.train.Venta_uni_hoy-FE.train.Dev_uni_proxima) ]

In [32]:
DemandaNotEqualVentaSubstractDev = DemandaNotEqualVentaSubstractDev.sort(columns=["Producto_ID","Cliente_ID"])
#demanda1Venta1Dev1.drop("Demanda_uni_equil",axis=1,inplace=True)

In [33]:
DemandaNotEqualVentaSubstractDev

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,log_Demanda_uni_equil
41695437,6,2281,7,3306,465617,41,0,0.000000,30,543.900024,0
62493519,8,2281,7,3306,465617,41,30,543.900024,70,1269.099976,0
9844037,3,2281,7,3306,685079,41,0,0.000000,9,163.169998,0
20816879,4,2281,7,3306,685079,41,0,0.000000,20,362.600006,0
9844368,3,2281,7,3309,1035265,41,0,0.000000,90,1631.699951,0
33946286,6,1147,4,1851,1399192,53,0,0.000000,1,14.300000,0
50035400,7,2013,1,5005,3439,72,0,0.000000,1,3.700000,0
55398938,8,1227,1,1614,8047,72,1,3.700000,3,11.100000,0
65812335,9,1227,1,1617,12277,72,2,7.400000,3,11.100000,0
7567893,3,2012,1,5010,13206,72,5,18.500000,12,44.400002,0


# Train include Test datas?

## Producto_ID Differences

In [75]:
testP = set(FE.test.Producto_ID)
trainP = set(FE.train.Producto_ID)

In [84]:
fileP = set(pd.read_csv("../../input/producto_tabla.csv").Producto_ID)

In [85]:
print len(testP)
print len(trainP)
print len(fileP)

1522
1799
2592


In [116]:
print trainP - fileP
print testP - fileP
print len(testP - trainP)
print testP - trainP

set([])
set([])
34
set([32026, 37404, 37405, 32798, 32421, 37745, 36524, 37618, 35246, 33053, 46131, 32820, 37688, 36673, 37702, 35191, 32591, 37202, 42323, 48217, 32224, 98, 31203, 37610, 31211, 46064, 37617, 37362, 37620, 31655, 37494, 37495, 37496, 37626])


## Cliente_ID Differences

In [92]:
testCl = set(FE.test.Cliente_ID)
trainCl = set(FE.train.Cliente_ID)
fileCl = set(pd.read_csv("../../input/cliente_tabla.csv").Cliente_ID)

In [115]:
print trainCl - fileCl
print testCl - fileCl
print len(testCl - trainCl)
#print testCl - trainCl

set([])
set([])
9663


## Agencia_ID Differences

In [94]:
testA = set(FE.test.Agencia_ID)
trainA = set(FE.train.Agencia_ID)
fileA = set(pd.read_csv("../../input/town_state.csv").Agencia_ID)

In [99]:
print trainA - fileA
print testA - fileA
print testA - trainA

set([])
set([])
set([])


## Ruta_SAK Differences

In [95]:
testR = set(FE.test.Ruta_SAK)
trainR = set(FE.train.Ruta_SAK)

In [114]:
print len(testR - trainR)
print testR - trainR
print len(trainR - testR)

17
set([9349, 3555, 3556, 3557, 3558, 7433, 7775, 8044, 4559, 9585, 6162, 3801, 9562, 3803, 8189, 6148, 1823])
1012


In [101]:
testCh = set(FE.test.Canal_ID)
trainCh = set(FE.train.Canal_ID)

In [102]:
print testCh - trainCh
print trainCh - testCh

set([])
set([])


## Producto_ID Cliente_ID

In [ ]:
testPC = pd.unique(FE.test[['Producto_ID', 'Cliente_ID']].values)

In [8]:
pd.unique(FE.test[['Producto_ID', 'Cliente_ID']].values)

1 loop, best of 3: 16.9 s per loop


In [11]:
testPCl = set(FE.test.groupby(['Producto_ID', 'Cliente_ID']).indices.keys())

In [12]:
trainPCl = set(FE.train.groupby(['Producto_ID', 'Cliente_ID']).indices.keys())
#trainPC = pd.unique(FE.train[['Producto_ID', 'Agencia_ID']].values)

In [16]:
print len(testPCl)
print len(trainPCl)
print len(testPCl-trainPCl)

6225450
25831538
1280296


In [91]:
set1 = set([1,2,3])
set2 = set([2,3,4])
print set1 | set2, set1-set2, set1 & set2

set([1, 2, 3, 4]) set([1]) set([2, 3])
